In [1]:
!pip install jovian --upgrade --quiet

In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
import pickle

In [3]:
df = pd.read_csv("output.csv")

In [4]:
pd.Categorical(df["Weather"])

['Sunny', 'Sunny', 'Sunny', 'Cool', 'Sunny', ..., 'Sunny', 'Sunny', 'Cool', 'Cool', 'Cool']
Length: 431686
Categories (5, object): ['Cool', 'Rainy', 'Sunny', 'ThunderStorm', 'Weather']

In [5]:
pd.Categorical(df["Day"])

['MONDAY', 'MONDAY', 'MONDAY', 'MONDAY', 'MONDAY', ..., 'SUNDAY', 'SUNDAY', 'SUNDAY', 'SUNDAY', 'SUNDAY']
Length: 431686
Categories (8, object): ['Day', 'FRIDAY', 'MONDAY', 'SATURDAY', 'SUNDAY', 'THURSDAY', 'TUESDAY', 'WEDNESDAY']

In [7]:
pd.Categorical(df["Station Name"])

['Adarsh Nagar', 'Adarsh Nagar', 'Adarsh Nagar', 'Adarsh Nagar', 'Adarsh Nagar', ..., 'Yamuna Bank', 'Yamuna Bank', 'Yamuna Bank', 'Yamuna Bank', 'Yamuna Bank']
Length: 431686
Categories (138, object): ['AIIMS', 'Adarsh Nagar', 'Akshardham', 'Anand Vihar', ..., 'Vidhan Sabha', 'Vishwa Vidyalaya', 'Welcome', 'Yamuna Bank']

In [8]:
def try_int(x):
    try:
        return int(x)
    except ValueError:
        return 

df["Weather"] = pd.Categorical(df["Weather"]).codes
df["Day"] = pd.Categorical(df["Day"]).codes
df["Station Name"] = pd.Categorical(df["Station Name"]).codes
df["Time"] = df["Time"].map(lambda x: try_int(x.split(":")[0]))
df["Population"] = df["Population"].map(lambda x: try_int(x))

In [9]:
x = df[["Station Name", "Day", "Time", "Weather"]].values
y = df[["Population"]].values

In [10]:
x = np.nan_to_num(x)
y = np.nan_to_num(y)
np.any(np.isnan(x))
# norm_x = preprocessing.scale(x)
# norm_y = preprocessing.scale(y)

False

In [11]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.35, random_state=100)

In [12]:
RandomForesRegModel = RandomForestRegressor(n_estimators = 6, random_state = 0)
RandomForesRegModel.fit(x_train.reshape(-1,4), y_train.ravel())

RandomForestRegressor(n_estimators=6, random_state=0)

In [13]:
y_pred = RandomForesRegModel.predict(x_test.reshape(-1,4))

In [14]:
y_pred[1:10].reshape(-1,1)

array([[1074.6615639 ],
       [  72.69947665],
       [1099.7677937 ],
       [  33.17528714],
       [1344.13888419],
       [ 108.78733974],
       [ 367.78580553],
       [ 206.57260101],
       [ 858.56593221]])

In [15]:
y_test[1:10].reshape(-1,1)

array([[1127.],
       [  52.],
       [1040.],
       [  16.],
       [1365.],
       [ 136.],
       [ 339.],
       [ 157.],
       [ 867.]])

In [16]:
print('Accuracy : ', r2_score(y_test, y_pred.reshape(-1,1))*100)
print('MSE: ', mean_squared_error(y_test, y_pred.reshape(-1,1)))

Accuracy :  98.19030927379211
MSE:  2214.8806788867537


In [17]:
PKL_filename = "RFR_Model.pkl"

with open(PKL_filename, 'wb') as file:
    pickle.dump(RandomForesRegModel, file)

In [35]:
with open ("RFR_Model.pkl", 'rb') as file:
    Loaded_Model = pickle.load(file)
Loaded_Model
test_input=np.array([1,6,10,2]) # "Station Name[0-137]", "Day[0-7]", "Time[10-8]", "Weather[0-4]"
result = Loaded_Model.predict(test_input.reshape(-1,4))
result[0]

1539.3043803418805

In [34]:
# score = Loaded_Model.score(x_test.reshape(-1,4),y_test)
# score